# Projekt PDU nr 3

### Michał Kukla 311134

Raport służy pokazaniu, jak wykonać polecenia SQL podane poniżej za pomocą:

1. funkcji pandas.read_sql_query("""SQL query""")

2. innych metod i funkcji z pakietu pandas


## Ramki danych

Będę pracował na uproszczonym zrzucie zanonimizowanych danych z serwisu <https://travel.stackexchange.com/>, który składa się z następujących ramek danych:

* Badges.csv.gz

* Comments.csv.gz

* PostLinks.csv.gz

* Posts.csv.gz

* Tags.csv.gz

* Users.csv.gz

* Votes.csv.gz


Znaczenie poszczególnych kolumn w podanych ramkach znajdziemy tutaj:  <https://ia600107.us.archive.org/27/items/stackexchange/readme.txt>.

Poniżej zaprezentuję podane polecenia w języku SQL oraz funkcje, które je wykonują (gdzie dokładniejszy opis, co robi funkcja w danym momencie, będzie się znajdował w komentarzach do kodu). W tym projekcie nie zawiorę wytłumaczenia działania kodu napisanego w SQL, ponieważ opis ten znajduje się w projekcie nr 2. 

Wynik funkcji z pakietu pandas będziemy porównywać z wynikiem funkcji pandas.read_sql_query("""SQL-owe zapytanie"""), ponieważ polega ona tylko na przepisaniu polecenia SQL, więc oczywiście jest poprawna.

### Załadowanie potrzebnych pakietów i ramek danych

Najpierw załadujemy potrzebne nam do zadań pakiety Pythona, a następnie zbiory danych.

In [1]:
import pandas as pd
import numpy as np
import os, os.path
import tempfile, sqlite3



# To nasz aktualny katalog roboczy
path = os.getcwd()

# Wczytujemy potrzebne ramki danych

Badges = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/Badges.csv.gz"), 
                        compression="gzip")

Comments = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/Comments.csv.gz"), 
                        compression="gzip")

PostLinks = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/PostLinks.csv.gz"), 
                        compression="gzip")

Posts = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/Posts.csv.gz"), 
                        compression="gzip")

Tags = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/Tags.csv.gz"), 
                        compression="gzip")

Users = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/Users.csv.gz"), 
                        compression="gzip")

Votes = pd.read_csv(os.path.join(path, 
                         "travel_stackexchange_com/Votes.csv.gz"), 
                        compression="gzip")


Pakiet **numpy** rozszerza zbiór typów danych znanych z Pythona, ułatwia analizę danych (np. tablice ndarray zapewniają wektoryzację).

Pakiet **pandas** (obok numpy) jest częścią składową biblioteki SciPy (scientific Python). Do najważniejszych klas w pandas należą:
* Series (reprezentuje poszczególne kolumny)
* DataFrame (reprezentuje dane tabelaryczne, gdzie kolumny to zmienne, a wiersze to obserwacje)
* Index (czyli obiekt wektoropodobny, który służy do etykietowania wierszy - w tym projekcie okaże się dla nas zbędny) 


Pakiet pandas jest dosyć podobny do pakietu dplyr w R (co będzie widoczne przy rozwiązaniach zadań).

Załączyliśmy pakiet os i os.path, które zapewniają funkcje do tworzenia i usuwania folderu (my dzięki nim odczytujemy pliki csv i zarządzamy ścieżką).

Pakietu tempfile użyliśmy, by przy zamykaniu programu usuwać dodatkowy wytworzony plik, na którym pracowaliśmya pakietu sqlite3 do obsługi bazy danych.
\
\
\
\
Teraz stworzymy bazę danych.

In [2]:
baza = os.path.join(tempfile.mkdtemp() , 'baza.db')

# Za każdym razem będziemy pracować z nową bazą danych, więc jak już jakaś istnieje w zadanym miejscu, to zostanie usunięta

if os.path.isfile(baza):
    os.remove(baza)
    
    
conn = sqlite3.connect(baza)  # otwieramy połączenie do bazy

I stworzymy tabele w bazie.

In [3]:
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

## Zadanie 1

Mamy podanie polecenie w języku SQL:

```{r}
SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC
```

a) rozwiązanie przy pomocy funkcji read_sql_query

In [4]:
df_sql_1 = pd.read_sql_query("""
SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC
""", conn)

df_sql_1

,Count,TagName
0,9470,visas
1,5119,usa
2,4601,uk
3,4460,air-travel
4,3503,customs-and-immigration
5,3296,schengen
6,2058,transit
7,1695,passports
8,1665,indian-citizens
9,1517,trains


b) rozwiązanie przy pomocy innych funkcji z pakietu pandas

In [5]:
x1 = Tags.loc[(Tags.Count > 1000), ["Count","TagName"]]  # z Tags wybieramy kolumny Count i TagName, gdzie Count > 1000
x1 = x1.sort_values(by = "Count", ascending = False)  # ustawiamy wiersze tak, by kolejne wartości z Count były malejąco
x1 = x1.reset_index()  # usuwamy index, czyli chcemy by kolejne wartości miały numery 0,1,2,3,... a nie 1,45,2,34,5,...
x1 = x1.drop("index", axis = 1)  # z x1 usuwamy kolumnę Index, która się wytwarza (Index jest "o współrzędnej jeden")
x1

,Count,TagName
0,9470,visas
1,5119,usa
2,4601,uk
3,4460,air-travel
4,3503,customs-and-immigration
5,3296,schengen
6,2058,transit
7,1695,passports
8,1665,indian-citizens
9,1517,trains


In [6]:
df_sql_1.equals(x1)

True

Otrzymaliśmy True, zatem napisana przez nas funkcja jest poprawna.

## Zadanie 2

Mamy kolejne polecenie w języku SQL:


```{r}
SELECT Location, COUNT(*) AS Count
FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
```

a) rozwiązanie przy pomocy funkcji read_sql_query

In [7]:
df_sql_2 = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn)

df_sql_2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


b) rozwiązanie przy pomocy innych funkcji z pakietu pandas

In [8]:
x2 = pd.merge(Users, Posts, left_on = 'Id', right_on='OwnerUserId', how = "inner")  # W tym miejscu wykonujemy inner join na Users i Posts, z czego łączymy te ramki po Id (z Users) i po OwnerUserId (z Posts)

x2 = x2.loc[:, ["OwnerUserId", "Id_x", "Location"]]                                 # z ramki x2 wybieramy 3 kolumny - Id_x zostało wytworzone po sklejeniu Id i OwnerUserId
x2 = x2.rename(columns = {"Id_x": "Id"})                                            # zmieniamy nazwę kolumny z "Id_x" na "Id"

# Połączyliśmy ramki Users i Posts oraz wybraliśmy z nich odpowiednie kolumny, przechodzimy do funkcji zewnętrznej

x2 = x2.loc[(x2.Location != "")].groupby("Location").size()                         # z ramki x2 bierzemy tylko te wiersze, gdzie lokacja nie jest "", grupujemy po lokacji, wykonując operację zliczania, ile jest unikatowych wartości z kolumny Location

x2 = x2.to_frame()                                                                  # x2 jest typem Series, zamienimy na DataFrame
x2 = x2.rename(columns = {0: "Count"})                                              # przy operacji grupowania i zliczania wartości (size()), nowa kolumna jest domyślnie nazywana jako 0. My zmieniamy jej nazwę na Count
x2 = x2.sort_values(by = "Count", ascending = False).reset_index().head(10)         # zmieniamy kolejność wierszy, by kolejne wartości z Count były ciągiem malejącym. Przenumerowujemy kolejne wiersze na 0,1,2,3,... (reset_index()) i bierzemy 10 pierwszych wierszy tabeli - head(10)
x2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [9]:
df_sql_2.equals(x2)  # True

True

## Zadanie 3

Mamy kolejne polecenie w języku SQL:


```{r}
SELECT Year, SUM(Number) AS TotalNumber
FROM (
    SELECT
        Name,
        COUNT(*) AS Number,
        STRFTIME('%Y', Badges.Date) AS Year
    FROM Badges
    WHERE Class = 1
    GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber
```

a) rozwiązanie przy pomocy funkcji read_sql_query

In [10]:
df_sql_3 = pd.read_sql_query("""
SELECT Year, SUM(Number) AS TotalNumber
FROM (
    SELECT
        Name,
        COUNT(*) AS Number,
        STRFTIME('%Y', Badges.Date) AS Year
    FROM Badges
    WHERE Class = 1
    GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber
""", conn)

df_sql_3

,Year,TotalNumber
0,2011,16
1,2012,23
2,2013,66
3,2021,153
4,2014,197
5,2020,265
6,2015,328
7,2016,509
8,2017,552
9,2018,697


b) rozwiązanie przy pomocy innych funkcji z pakietu pandas

In [11]:
x3 = Badges.loc[Badges.Class == 1].loc[:, ["Name","Date", "Class"]]       # z ramki Badges wybieramy te wiersze, w których wartość z kolumny Class wynosi 1. Wybieramy 3 wiersze z otrzymanej ramki
x3['Date'] = pd.to_datetime(x3['Date'])                                   # zmieniamy format wartości kolumny Date na datetime
x3.Date = x3.Date.dt.strftime("%Y")                                       # zmieniamy wartości z Date - z dokładnego czasu zostawiamy sam rok
x3 = x3.rename(columns = {"Date": "Year"})                                # zmieniamy nazwę kolumny - z "Date" na "Year"
x3 = x3.groupby(["Name", "Year"]).size().reset_index()                    # zliczamy wystąpienia unikatowych kombinacji wartości Name i Year, wiersze otrzymanej ramki numerujemy 0,1,2,3,... 
x3 = x3.rename(columns = {0: "Number"})                                   # jak było powiedziane w zadaniu poprzednim, domyślnie nazwaną kolumnę 0 nazwiemy "Number"

# Przechodzimy do funkcji zewnętrznej

x3 = x3.groupby(["Year"])["Number"].agg([np.sum])                         # grupujemy po roku, wykonując na Number operację sumy (czyli gdy dany rok wystąpił 10 razy z wartościami 5, to sumujemy 5+5+...+5 = 50 w nowej kolumnie)
x3 = x3.rename(columns = {"sum": "TotalNumber"})                          # tutaj kolumna jest nazywana domyślnie "sum", my tę nazwę zmieniamy na "TotalNumber"
x3 = x3.sort_values(by = "TotalNumber", ascending = True).reset_index()   # zmieniamy kolejność wierszy, by kolejne wartości z TotalNumber były rosnąco, resetujemy index (dokładne wytłumaczenie, co to oznacza, znajduje się powyżej)
x3

,Year,TotalNumber
0,2011,16
1,2012,23
2,2013,66
3,2021,153
4,2014,197
5,2020,265
6,2015,328
7,2016,509
8,2017,552
9,2018,697


In [12]:
df_sql_3.equals(x3)  # True

True

## Zadanie 4

Oto czwarte polecenie w języku SQL:


```{r}
SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
    SELECT
        AnsCount.AnswersCount,
        Posts.Id,
        Posts.OwnerUserId
    FROM (
            SELECT Posts.ParentId, COUNT(*) AS AnswersCount
            FROM Posts
            WHERE Posts.PostTypeId = 2
            GROUP BY Posts.ParentId
          ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC, AccountId ASC
LIMIT 10

```

a) rozwiązanie przy pomocy funkcji read_sql_query

In [13]:
df_sql_4 = pd.read_sql_query("""
SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
    SELECT
        AnsCount.AnswersCount,
        Posts.Id,
        Posts.OwnerUserId
    FROM (
            SELECT Posts.ParentId, COUNT(*) AS AnswersCount
            FROM Posts
            WHERE Posts.PostTypeId = 2
            GROUP BY Posts.ParentId
          ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC, AccountId ASC
LIMIT 10
""", conn)

df_sql_4

,AccountId,DisplayName,Location,AverageAnswersCount
0,280.0,csmba,"San Francisco, CA",11.0
1,40811.0,vocaro,"San Jose, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,20473.0,Jeremy Boyd,"Houston, TX",8.0
7,42364.0,Petrogad,None,8.0
8,54571.0,Christian,None,8.0
9,79346.0,Thomas Matthews,California,8.0


b) rozwiązanie przy pomocy innych funkcji z pakietu pandas

In [14]:
x4 = Posts.groupby("ParentId").size().reset_index()                                              # Grupujemy po ParentId, zliczając ilość wystąpień. Przywracamy index do postaci 0,1,2,3,... (resetujemy index)
x4 = x4.rename(columns = {0: "AnswersCount"})                                                    # Zmieniamy nazwę wytworzonej kolumny na "AnswersCount"
AnsCount = x4

# Mamy ramkę AnsCount, przechodzimy do bardziej zewnętrznej funkcji

x4 = pd.merge(AnsCount, Posts, left_on = "ParentId", right_on = "Id", how = "inner")             # wykonujemy inner join - łączymy ramki AnsCount i Posts sklejając kolumny w miejscach, gdzie wystąpiły te same wartości w kolumnie ParentId i Id (łaczymy te kolumny w jedną). Inne wiersze zostaną pominięte (stąd 'inner' join)
x4 = x4.reset_index()                                                                            # resetujemy index
x4 = x4.loc[:, ["AnswersCount", "Id", "OwnerUserId"]]                                            # z x4 wybieramy kolumny AnswersCount, Id i OwnerUserId
PostAuth = x4

# Mamy ramkę PostAuth, przechodzimy do funkcji najbardziej zewnętrznej

x4 = pd.merge(PostAuth, Users, left_on = "OwnerUserId", right_on = "AccountId", how = "inner")   # wykonujemy inner join na PostAuth i Users, łącząc OwnerUserId z AccountId
inne = x4.groupby(["OwnerUserId"])["AnswersCount"].agg([np.mean])                                # grupujemy po OwnerUserId, wykonując operację średniej na wartościach z AnswersCount
inne = inne.rename(columns = {"mean": "AnswersCount"})                                           # zmieniamy domyślną nazwę kolumny z "mean" na "AnswersCount"
x4 = pd.merge(x4, inne, on = "OwnerUserId", how = "inner")                                       # łączymy ramki danych - w skrócie ta i powyższe 2 wiersze służą do zgrupowania względem OwnerUserId (więc w otrzymanej ramce mają też być "AccountId", "DisplayName" i "Location", ale wykonujemy operację średniej arytmetycznej tylko na kolumnie "AnswersCount")
x4 = x4.loc[:, ["AccountId", "DisplayName", "Location", "AnswersCount_y"]]                       # wybieramy 4 wiersze
x4 = x4.rename(columns = {"AnswersCount_y": "AverageAnswersCount"})                              # zmieniamy nazwę kolumny
x4 = x4.drop_duplicates(subset = ["AccountId"])                                                  #usuwam zdublowany wiersz z AccountId - wiem, że taki jest z poprzedniej pracy domowej
x4 = x4.sort_values(by = ["AverageAnswersCount","AccountId"], ascending = [False, True])         # szeregujemy po AverageAnswersCount malejąco. Jeżeli kilka razy wystąpiła ta sama wartość w tej kolumnie, to szeregujemy te po wartości z AccountId rosnąco
x4 = x4.reset_index().drop("index", axis = 1).head(10)                                           # usuwamy kolumnę Index i wybieramy 10 pierwszych wierszy z otrzymanej ramki
x4

,AccountId,DisplayName,Location,AverageAnswersCount
0,280.0,csmba,"San Francisco, CA",11.0
1,40811.0,vocaro,"San Jose, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,20473.0,Jeremy Boyd,"Houston, TX",8.0
7,42364.0,Petrogad,NaN,8.0
8,54571.0,Christian,NaN,8.0
9,79346.0,Thomas Matthews,California,8.0


In [15]:
df_sql_4.equals(x4)  # True

True

## Zadanie 5

Piąte polecenie w języku SQL:


```{r}
SELECT Posts.Title, Posts.Id,
        STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
        VotesByAge.Votes
FROM Posts
JOIN (
        SELECT
            PostId,
            MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
            MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
            SUM(Total) AS Votes
        FROM (
            SELECT
                PostId,
                CASE STRFTIME('%Y', CreationDate)
                    WHEN '2021' THEN 'new'
                    WHEN '2020' THEN 'new'
                    ELSE 'old'
                    END VoteDate,
                COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId IN (1, 2, 5)
            GROUP BY PostId, VoteDate
        ) AS VotesDates
        GROUP BY VotesDates.PostId
        HAVING NewVotes > OldVotes
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('')
ORDER BY Votes DESC
LIMIT 10

```

a) rozwiązanie przy pomocy funkcji read_sql_query

In [16]:
df_sql_5 = pd.read_sql_query("""
SELECT Posts.Title, Posts.Id,
        STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
        VotesByAge.Votes
FROM Posts
JOIN (
        SELECT
            PostId,
            MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
            MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
            SUM(Total) AS Votes
        FROM (
            SELECT
                PostId,
                CASE STRFTIME('%Y', CreationDate)
                    WHEN '2021' THEN 'new'
                    WHEN '2020' THEN 'new'
                    ELSE 'old'
                    END VoteDate,
                COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId IN (1, 2, 5)
            GROUP BY PostId, VoteDate
        ) AS VotesDates
        GROUP BY VotesDates.PostId
        HAVING NewVotes > OldVotes
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('')
ORDER BY Votes DESC
LIMIT 10
""", conn)
# 3217 rows
df_sql_5

,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64
9,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64


b) rozwiązanie przy pomocy innych funkcji z pakietu pandas

In [17]:
a = Votes.VoteTypeId                                                                   # dla skrócenia zapisu, zmienna pod zmienną a przypiszemy kolumę z VoteTypeId
x5 = Votes.loc[(a == 1) | (a == 2) | (a == 5)]                                         # z Votes wybierzemy te wartości, gdzie a ==1 lub a == 2 lub a == 5 (pandas w specyficzny sposób odczytuje rzeczy, np. warunki muszą być w nawiasach i zamiast pisać "or" piszemy "|" - bp inaczej funkcja nie zadziała poprawnie)
a = pd.to_datetime(x5.CreationDate).dt.strftime("%Y")                                  # pod zmienną a przypiszemy kolumnę CreationDate, której wartości najpierw zamieniamy na datetime, a potem wydobywamy z nich tylko rok
x5 = x5.assign(VoteDate = a).loc[:, ["PostId", "VoteDate"]]                            # dodajemy kolumnę o wartościach z a, nazywamy ją "VoteDate". Z ramki x5 wybieramy tylko kolumny "PostId" i "VoteDate" 
x5.loc[(x5['VoteDate'] == "2021") | (x5['VoteDate'] == "2020"), "VoteDate"] = "new"    # x5['VoteDate'] to to samo co x5.VoteDate. Wartości z VoteDate równe 2020 lub 2021 zamieniamy na "new"
x5.loc[(x5['VoteDate'] != "new"), "VoteDate"] = "old"                                  # wartości z VoteDate różne od 2020 i 2021 (różne od "new") zamieniamy na "old"
x5 = x5.groupby(["PostId", "VoteDate"]).size().reset_index()                           # grupujemy po 2 kolumnach, zliczając ile wystąpiło unikalnych kombinacji wartości z tych kolumn, następnie resetujemy index
x5 = x5.rename(columns = {0: "Total"}).sort_values(by = "PostId", ascending = True)    # zmieniamy nazwę kolumny z 0 na "Total", ustawiamy wiersze tak, by kolejne wartości z PostId były uporządkowane rosnąco
x5 = x5.sort_values(by = ["PostId", "VoteDate"], ascending = [True,True])              # wykonujemy tu dodatkową operację szeregowania wierszy (nie tylko po PostId, ale też na VoteDate), by ta funkcja działała tak samo, jak zagnieżdżona SQLowa funkcja - przy szeregowaniu wierszy rosnąco i kiedy 2 wartości się powtarzają, to funkcja sqlowa szereguje wtedy te 2 wartości po innym parametrze
VotesDates = x5
# Mamy już VotesDates, przechodzimy do funkcji bardziej zewnętrznej
a = x5.VoteDate
x5 = x5.assign(NewVotes = a).loc[:, ["PostId","NewVotes","VoteDate","Total"]]          # dodajemy nową kolumnę z wartościami z a i nazywamy ją "VotesDates", z otrzymanej ramki wybieramy 4 kolumny
x5 = x5.rename(columns = {"VoteDate": "OldVotes"})                                     # zmieniamy nazwę kolumny na "OldVotes"
x5.NewVotes = np.where(x5.NewVotes == "new", x5.Total, 0)                              # tam, gdzie w "NewVotes" występuje wartość "new", dajemy wartość z z kolumny Total z tego samego wiersza. W przeciwnym wypadku wstawiamy 0
x5.OldVotes = np.where(x5.OldVotes == "old", x5.Total, 0)                              # analogicznie dla "OldVotes" - gdzie wartość "old", to wpisujemy wartość z tego samego wiersza kolumny Total, a wartości "new" zmieniamy na 0
a = x5.groupby(["PostId"])["NewVotes","OldVotes"].agg(np.max)                          # grupujemy po PostId, na NewVotes i OldVotes wykonując operację maksimum - pojawia nam się warning, który nie wpływa na wynik funkcji po naszych kolejnych działaniach poniższych
b = x5.groupby(["PostId"])["Total"].agg(np.sum)                                        # grupujemy po PostId, wykonując sumowanie wartości z Total
x5 = pd.merge(a,b, on = "PostId", how = "inner")                                       # mergujemy ramki a i b (inner join). W skrócie ta i 2 powyższe linijki kodu były w celu pogrupowania względem PostId, wykonując na NewVotes i OldVotes funkcję maksimum, a na Total funkcji sumowania
x5 = x5[x5.NewVotes > x5.OldVotes].rename(columns = {"Total": "Votes"}).reset_index()  # bierzemy te wiersze, gdzie wartość z NewVotes jest większa od OldVotes, zmieniamy nazwę kolumny "Total" na "Votes" i resetujemy index
VotesByAge = x5
# Mamy już VotesByAge, zostaje nam funkcja najbardziej zewnętrzna
x5 = pd.merge(VotesByAge, Posts, left_on = "PostId", right_on = "Id", how = "inner")   # mergujemy ramkę VotesByAge z Posts po PostId i Id
a = pd.to_datetime(x5.CreationDate).dt.strftime("%Y-%m-%d")                            # pod zmienną a wpisujemy wartości z kolumny CreationDate (z x5), gdzie wartości są postaci datetime. Następnie te wartości modyfikujemy tak, by były postaci "rok-miesiąc-dzień"
x5 = x5.assign(Date = a).loc[:, ["Title","Id","Date","Votes"]]                         # dodajemy nową kolumnę z wartościami z a, nazywamy ją Date i w naszej ramce zostawiamy 4 kolumny
x5 = x5.loc[x5.Title != ""].dropna()                                                   # zostawiamy w ramce te wiersze, gdzie wartość z Title nie jest pusta (postaci ""), a następnie usuwamy jeszcze wiersze z wartościami NaN
x5 = x5.sort_values(by = ["Votes","Date"], ascending = [False,True]).reset_index()     # sortujemy po Votes rosnąco, a jeżeli niektóre wartości z Votes się powtarzają, to dla nich wiersze szeregujemy rosnąco względem wartości z Date (by wynik był taki sam jak funkcji SQLowej)
x5 = x5.drop("index", axis = 1)                                                        # usuwamy Index - obiekt wektoropodobny do etykietowania wierszy
x5 = x5.head(10)                                                                       # w ramce x5 zostawiamy tylko 10 pierwszych wierszy
x5

<ipython-input-17-da13b176a81f>:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  a = x5.groupby(["PostId"])["NewVotes","OldVotes"].agg(np.max)                          # grupujemy po PostId, na NewVotes i OldVotes wykonując operację maksimum - pojawia nam się warning, który nie wpływa na wynik funkcji po naszych kolejnych działaniach poniższych


,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64
9,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64


In [18]:
df_sql_5.equals(x5)  # True

True

Wszystkie nasze funkcje napisane za pomocą "bazowych" funkcji z pakietu pandas dają ten sam wynik, co funkcja pd.read_sql_query(), ponieważ po wywołaniu funkcji equals za każdym razem otrzymaliśmy True. Przejdziemy zatem do wniosków, ale najpierw po zakończeniu pracy z zadaniami zamkniemy połączenie do bazy danych.

In [19]:
conn.close()

## Wnioski

* Zauważyłem wiele podobieństw funkcji z pandas do pakietu dplyr i data.table z R. Funkcje są podobne i bardzo często korzystamy z nawiasów kwadratowych. Kropka (".") często pełni funkcję w pandas taką, jak tzw. "pipe" w dplyr ("%>%") - stąd funkcję wywołującą zapytanie SQLowe można napisać nawet w 1 wierszu, lecz wtedy kod byłby mniej czytelny
* Moim zdaniem, porównując funkcje napisane w base, dplyr, data.table i pandas, najtrudniej było napisać funkcję korzystając tylko z pakietu pandas. Jest on dosyć intuicyjny, lecz bardzo często przy pisaniu kodu napotykałem się z licznymi błędami i warningami 
(np. " a==1 | a==5 " było odczytywane inaczej, niż" (a==1)|(a==5) ", a także inaczej niż " (a==1) or (a==5) ").
* Po wykonaniu tych samych zadań w różnych pakietach stwierdzam, że na ten moment moim zdaniem język R jest wygodniejszy do analizy danych. Funkcje w nim napisane częściej "rozpoznawały zamiary osoby piszącej kod" i pokazywały mniej warningów. Zatem nauka pakietu pandas była najtrudniejsza, ponieważ wymagała uważniejszego przeczytania dokumentacji.